# Audio curation in NeMo Curator

In the following notebook, we'll be exploring basic functionality that NeMo Curator has for audio dataset curation. NeMo Curator has a few built-in modules for:

- Download and ManifestFleursStage

- ASR (Autimatic Speech Recognition) Inference using NeMo toolkit

- Calculate pairwise WER (Word Error Rate)

- Get duration for each audio 

- Save metadata to Jsonl file

We'll cover all the modules in this tutorial notebook. First, we'll need to install NeMo toolkit and NeMo Curator!

NOTE: Please ensure you meet the requirements before proceeding!